In [ ]:
import os
import nest_asyncio
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings


nest_asyncio.apply()
openai_api_key = os.getenv("OPENAI_API_KEY")

openai_api_key

In [ ]:
from app.utils.llm.helpers import init_llm_configurations
from app.settings import Constants


init_llm_configurations(llm_model=Constants.LLM_MODEL, embedding_model=Constants.EMBEDDING_MODEL)

In [ ]:
documents = SimpleDirectoryReader("../examples").load_data(show_progress=True)
documents

In [9]:
# Build index
index = VectorStoreIndex(documents)

In [34]:
SYSTEM_PROMPT = """
You are a human resources professional at a company that needs to quickly find information in its policy documents.

Guidelines:
- Provide information explicitly stated in policy documents
- For basic contextual questions (company name, policy type, etc), use ONLY information directly visible in the provided context
- If you cannot find an answer, please output "Không tìm thấy thông tin, hãy liên hệ HR. SĐT: 0123456789 hoặc email contact.hr@company.com."
- No assumptions or interpretations about policy details
- Do not explain
- Your primary language is Vietnamese

Let's work this out in a step by step way to be sure we have the right answer."""

CONTEXT_PROMPT = """
The following is a friendly conversation between an employee and a human resources professional.
The professional is talkative and provides lots of specific details from her context.
If the professional does not know the answer to a question, she truthfully says she does not know.

Here are the relevant documents for the context:

{context_str}

## Instruction
Based on the above documents, provide a detailed answer for the employee question below.
Answer "don't know" if not present in the document."""

CONTEXT_REFINE_PROMPT = """
The following is a friendly conversation between an employee and a human resources professional.
The professional is talkative and provides lots of specific details from her context.
If the professional does not know the answer to a question, she truthfully says she does not know.

Here are the relevant documents for the context:

{context_msg}

Existing Answer:

{existing_answer}

## Instruction
Refine the existing answer using the provided context to assist the user.
If the context isn't helpful, just repeat the existing answer and nothing more."""

CONDENSE_PROMPT = """
Given the following conversation between an employee and a human resources professional and a follow up question from the employee.
Your task is to firstly summarize the chat history and secondly condense the follow up question into a standalone question.

Chat History:
'''
{chat_history}
'''

Follow Up Input:
'''
{question}
'''

Output format: a standalone question.

Your response:"""

In [35]:
# Create retriever and the chat engine
from llama_index.core.chat_engine import CondensePlusContextChatEngine
from llama_index.core.postprocessor import LLMRerank
from llama_index.core.memory import BaseMemory


retriever = index.as_retriever()
memory = BaseMemory.from_defaults()

In [36]:
chat_engine = CondensePlusContextChatEngine.from_defaults(
    retriever=retriever,
    memory=memory,
    system_prompt=SYSTEM_PROMPT,
    context_prompt=CONTEXT_PROMPT,
    context_refine_prompt=CONTEXT_REFINE_PROMPT,
    node_postprocessors=[LLMRerank(top_n=5)],
    verbose=True
)

In [ ]:
response = chat_engine.chat("trường hợp nghỉ việc riêng không lương?")

print(f"Response: {response.response}")
print(f"Sources: {response.sources}")
print(f"Source nodes: {response.source_nodes}")